In [ ]:
import numpy as np
import pandas as pd
from math import floor, ceil
from numpy.linalg import cholesky, inv, solve 
from scipy.linalg import cho_solve
from scipy.stats import wishart, invwishart, gamma
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import calibration_and_holdout_data, summary_data_from_transaction_data
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases, plot_period_transactions
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [ ]:
def load_dataset(datafile, parse_dates=None):
    df = pd.read_csv(datafile, delimiter=',', parse_dates=parse_dates)
    return df


## Implementation

In [ ]:
# x ==> number of repeat purchases
# t ==> First purchase to last purchase
# T ==> First purchase to end of observation period


In [ ]:
# Setup Regressors (Covariates) for location of 1st-stage prior, i.e. beta = [log(lambda), log(mu)]
def set_regressors(data, covariates=[]):
    data['intercept'] = 1.0
    covariates = ['intercept'] + covariates
    covars = np.matrix(data[covariates])
    K = len(covariates)
    return covariates, covars, K

def get_diag(shape, val):
    d = np.zeros(shape=shape)
    np.fill_diagonal(d, val) 
    return d

def get_map_from_array(x):
    a_map = {}
    count = 0
    for val in x:
        a_map[val] = count
        count += 1
    return a_map

# set hyper priors "log_lambda", "log_mu"
def set_hyperpriors(K):  
    beta_0 = np.zeros(shape=(K, 2))
    A_0 = get_diag(shape=(K, K), val=0.01) # diffuse precision matrix
    # set diffuse hyper-parameters for 2nd-stage prior of gamma_0; follows defaults from rmultireg example
    nu_00 = 3 + K  # 30
    gamma_00 = get_diag(shape=(2, 2), val=nu_00) # diffuse precision matrix
    hyper_prior = {'beta_0': beta_0, 'A_0':A_0, 'nu_00':nu_00, 'gamma_00':gamma_00}
    return hyper_prior

def draw_z(data, level_1, level_1_params_map):
    tx = data['t_cal']
    Tcal = data['T_cal']
    p_lambda = level_1[level_1_params_map['lambda'], ]
    p_mu = level_1[level_1_params_map['mu'], ]
    mu_lam = p_mu + p_lambda
    t_diff = Tcal - tx
    prob = 1 / (1 + (p_mu / mu_lam) * (np.exp(mu_lam * t_diff) - 1))
    z = (np.random.uniform(size=len(prob)) < prob)
    z[z == True] = 1
    z = z.astype(int)
    return list(z.values)

def draw_tau(data, level_1, level_1_params_map):
    N = len(data)
    tx = data['t_cal']
    Tcal = data['T_cal']
    p_lambda = level_1[level_1_params_map['lambda'], ]
    p_mu = level_1[level_1_params_map['mu'], ]

    mu_lam = p_mu + p_lambda
    z = level_1[level_1_params_map['z'], ]

    alive = (z == 1)
    tau = np.zeros(shape=(N))

    # Case: still alive - left truncated exponential distribution -> [T.cal, Inf]
    if (np.sum(alive) > 0):
        tau[alive] = Tcal[alive] + np.random.exponential(scale=1.0/p_mu[alive], size=np.sum(alive))

    # Case: churned - double truncated exponential distribution -> [tx, T.cal]
    if (np.sum(~alive) > 0):
        mu_lam_tx = np.minimum(700, mu_lam[~alive] * tx[~alive])
        mu_lam_Tcal = np.minimum(700, mu_lam[~alive] * Tcal[~alive])
        rand = np.random.uniform(size=np.sum(~alive))        
        tau[~alive] = (-1.0 * np.log((1.0 - rand) * np.exp(-1.0 * mu_lam_tx) + rand * np.exp((-1.0 * mu_lam_Tcal)))) / mu_lam[~alive]

    return tau

def chol2inv(chol):
    return cho_solve((chol, False), np.eye(chol.shape[0])) 

def draw_wishart(df, scale):
    W = wishart.rvs(df, scale)
    IW = inv(W)
    C = cholesky(W).T
    CI = inv(C)
    return W, IW, C, CI

def rmultireg(Y, X, Bbar, A, nu, V):
    # standard multi-variate normal regression update
    # Slide 33 in http://ice.uchicago.edu/2008_presentations/Rossi/ICE_tutorial_2008.pdf
    n = Y.shape[0]
    m = Y.shape[1]
    k = X.shape[1]    

    RA = cholesky(A)
    W = np.concatenate((X, RA), axis=0) 
    Z = np.concatenate((Y, RA*Bbar), axis=0)
    IR = solve(np.triu(cholesky(np.dot(W.T, W)).T), np.eye(k,k)) #trimatu interprets the matrix as upper triangular and makes solve more efficient
    Btilde = np.dot(np.dot(IR, IR.T), np.dot(W.T,Z))
    E = Z - np.dot(W, Btilde)
    S = np.dot(E.T, E)
    W, IW, C, CI = draw_wishart(df=nu+n, scale=chol2inv(cholesky(V+S).T))
    samples = np.random.normal(size=k*m).reshape(k,m)
    B = Btilde + np.dot(IR, np.dot(samples, CI.T))
    return {'beta': B.T, 'gamma':IW}

def draw_level_2(covars, level_1, level_1_params_map, hyper_prior):
    # standard multi-variate normal regression update
    Y = np.log(level_1[[level_1_params_map['lambda'], level_1_params_map['mu']],].T)
    X = covars
    Bbar = hyper_prior['beta_0']
    A = hyper_prior['A_0']
    nu = hyper_prior['nu_00']
    V = hyper_prior['gamma_00']
    
    return rmultireg(Y, X, Bbar, A, nu, V)

def log_post(log_theta, mvmean, x, z, Tcal, tau, inv_gamma):
    log_lambda = log_theta[0,:] 
    log_mu = log_theta[1,:]
    diff_theta = np.subtract(log_theta, mvmean.T)
    diff_lambda = diff_theta[0,:]
    diff_mu = diff_theta[1,:]
    likel = (x * log_lambda) + ((1 - z) * log_mu) - (((z * Tcal) + (1 - z) * tau) * (np.exp(log_lambda) + np.exp(log_mu)))
    prior = -0.5 * ((np.square(diff_lambda) * inv_gamma[0, 0]) + (2 * np.multiply(diff_lambda, diff_mu) * inv_gamma[0, 1]) + (np.square(diff_mu) * inv_gamma[1, 1]))
    post = np.add(likel[0], prior)    
    post[0,log_mu > 5] = np.NINF  # cap !!
    return post

def step(cur_log_theta, cur_post, gamma, N, mvmean, x, z, Tcal, tau, inv_gamma):
    new_log_theta = cur_log_theta + np.vstack((gamma[0, 0] * np.random.standard_t(df=3, size=N), gamma[1, 1] * np.random.standard_t(df=3, size=N)))
    new_log_theta[0,:] = np.maximum(np.minimum(new_log_theta[0,:], 70), -70)
    new_log_theta[1,:] = np.maximum(np.minimum(new_log_theta[1,:], 70), -70)
    new_post = log_post(new_log_theta, mvmean, x, z, Tcal, tau, inv_gamma)
    # accept/reject new proposal
    mhratio = np.exp(new_post - cur_post)
    unif = np.random.uniform(size=N)
    accepted = np.asarray(mhratio > unif)[0]
    cur_log_theta[:,accepted] = new_log_theta[:, accepted]
    cur_post[0,accepted] = new_post[0,accepted]
    return {'cur_log_theta':cur_log_theta, 'cur_post':cur_post}

def draw_level_1(data, covars, level_1, level_1_params_map, level_2):
    # sample (lambda, mu) given (z, tau, beta, gamma)
    N = len(data)
    x = data['x_cal']
    Tcal = data['T_cal']
    z = level_1[level_1_params_map['z'], ]
    tau = level_1[level_1_params_map['tau'], ]
    mvmean = np.dot(covars, level_2['beta'].T)
    gamma = level_2['gamma']
    inv_gamma = inv(gamma)
    
    cur_lambda = level_1[level_1_params_map['lambda'], ]
    cur_mu = level_1[level_1_params_map['mu'], ]

    # current state
    cur_log_theta = np.vstack((np.log(cur_lambda), np.log(cur_mu)))
    cur_post = log_post(cur_log_theta, mvmean, x, z, Tcal, tau, inv_gamma)
    
    iter = 1  # how high do we need to set this? 1/5/10/100?
    for i in range(0, iter):
        draw = step(cur_log_theta, cur_post, gamma, N, mvmean, x, z, Tcal, tau, inv_gamma)
        cur_log_theta = draw['cur_log_theta']
        cur_post = draw['cur_post']

    cur_theta = np.exp(cur_log_theta)

    return {'lambda':cur_theta[0,:], 'mu':cur_theta[1,:]}

def run_single_chain(data, covariates, K, hyper_prior, nsample, nburnin, nskip):
    ## initialize arrays for storing draws ##
    LOG_LAMBDA = 0
    LOG_MU = 1
    nr_of_cust = len(data)
    #nr_of_draws = nburnin + nsample * nskip
    nr_of_draws = nburnin + nsample

    # The 4 is for "lambda", "mu", "tau", "z"
    level_1_params_map = get_map_from_array(['lambda', 'mu', 'tau', 'z'])
    level_1_draws = np.zeros(shape=(nsample, 4, nr_of_cust))

    level_2_draws = np.zeros(shape=(nsample, (2*K)+3))
    nm = ['log_lambda', 'log_mu']
    if (K > 1):
        nm = ['{}_{}'.format(val2, val1) for val1 in covariates for val2 in nm]
    nm.extend(['var_log_lambda', 'cov_log_lambda_log_mu', 'var_log_mu'])
    level_2_params_map = get_map_from_array(nm)
        
    ## initialize parameters ##
    data['t_cal_tmp'] = data['t_cal']
    data['t_cal_tmp'][data.t_cal == 0] = data['T_cal'][data.t_cal == 0] 
    level_1 = level_1_draws[1,]
    x_cal_mean = np.mean(data['x_cal'])
    t_cal_tmp_mean = np.mean(data['t_cal_tmp'])
    level_1[level_1_params_map['lambda'], ] = x_cal_mean/t_cal_tmp_mean
    level_1[level_1_params_map['mu'], ] = 1 / (data['t_cal'] + 0.5 / level_1[level_1_params_map['lambda'], ])
    
    ## run MCMC chain ##
    hyper_prior['beta_0'][0, LOG_LAMBDA] = np.log(np.mean(level_1[level_1_params_map['lambda'], ]))
    hyper_prior['beta_0'][0, LOG_MU] = np.log(np.mean(level_1[level_1_params_map['mu'], ]))
    
    for i in range(0, nr_of_draws):
        # draw individual-level parameters
        level_1[level_1_params_map['z'], ] = draw_z(data, level_1, level_1_params_map)
        level_1[level_1_params_map['tau'], ] = draw_tau(data, level_1, level_1_params_map)

        level_2 = draw_level_2(covars, level_1, level_1_params_map, hyper_prior)
        draw = draw_level_1(data, covars, level_1, level_1_params_map, level_2)
        level_1[level_1_params_map['lambda'], ] = draw["lambda"]
        level_1[level_1_params_map['mu'], ] = draw["mu"]
        
        #nk = int(round((i - nburnin) / nskip))        
        if (i >= nburnin):
            #Store
            idx = i - nburnin
            level_1_draws[idx,:,:] = level_1 # nolint
            level_2_draws[idx,:] = list(np.array(level_2['beta'].T).reshape(-1)) + [level_2['gamma'][0, 0], level_2['gamma'][0, 1], level_2['gamma'][1,1]]
        if (i % 100) == 0:
            print('draw: {}'.format(i))
    coeff_mean = np.mean(level_2_draws, axis=0)
    coeff_stddev = np.std(level_2_draws, axis=0)    
    coeff = {}
    for param in level_2_params_map:
        coeff[param] = {}
        coeff[param]['mean'] = coeff_mean[level_2_params_map[param]]
        coeff[param]['stddev'] = coeff_stddev[level_2_params_map[param]]
    
    return {"level_1":level_1_draws, "level_1_params_map":level_1_params_map
          , "level_2":level_2_draws, "level_2_params_map":level_2_params_map
          , "coeff": coeff}    

####MCMC Functions
def get_correlation(draws):
    l2pmap = draws["level_2_params_map"]
    draw_means = np.mean(draws['level_2'], axis=0)
    corr = draw_means[l2pmap['cov_log_lambda_log_mu']]/(np.sqrt(draw_means[l2pmap['var_log_lambda']]) * np.sqrt(draw_means[l2pmap['var_log_mu']]))
    return corr

def get_nr_of_cust(draws):
    nr_of_cust = draws["level_1"].shape[2]
    return nr_of_cust

def PAlive(draws):
    l1pmap = draws["level_1_params_map"]
    nr_of_cust = get_nr_of_cust(draws)
    p_alive = np.mean(draws["level_1"][:,l1pmap['z'],:], axis=1)
    return p_alive

def draw_left_truncated_gamma(lower, k, lamda):
    pg = gamma.cdf(x=lower, a=k, scale=1.0/(k*lamda))
    rand = np.random.uniform(1, pg, 1)
    qg = gamma.ppf(q=rand, a=k, scale=1.0/(k*lamda))
    return qg

def DrawFutureTransactions(data, draws, sample_size=None):
    nr_of_draws = draws["level_2"].shape[0]
    if sample_size is not None:
            nr_of_draws = sample_size
    nr_of_cust = get_nr_of_cust(draws)
    parameters = draws["level_1_params_map"]
    x_holdout = np.zeros(shape=(nr_of_draws, nr_of_cust))
    t_cal = data['t_cal']
    T_holdout = data['T_holdout']
    T_cal = data['T_cal']

    for i in range(0, nr_of_cust):
        print('...processing customer: {} of {}'.format(i, nr_of_cust))
        Tcal = T_cal[i]
        Tholdout = T_holdout[i]
        tcal = t_cal[i]
        taus = draws['level_1'][:,parameters['tau'],i]
        ks = np.ones(shape=(len(taus)))    
        lamdas = draws['level_1'][:,parameters['lambda'],i]
        if sample_size is not None:
            taus = taus[sample_size]
            ks = ks[sample_size]
            lambdas = lambdas[sample_size]

        alive = taus > Tcal
        # Case: customer alive
        idx = 0
        for alive_val in alive:
            if alive_val:
                # sample itt which is larger than (Tcal-tx)
                itts = draw_left_truncated_gamma(Tcal - tcal, ks[idx], lamdas[idx])
                # sample 'sufficiently' large amount of inter-transaction times
                minT = np.minimum(Tcal + Tholdout - tcal, taus[idx] - tcal)
                nr_of_itt_draws = int(np.maximum(10, np.round(minT * lamdas[idx])))
                itts = np.hstack((itts, np.array(gamma.rvs(a=ks[idx], loc=ks[idx]*lamdas[idx], size=nr_of_itt_draws*2))))
                if (np.sum(itts) < minT):
                    itts = np.hstack((itts, np.array(gamma.rvs(a=ks[idx], loc=ks[idx]*lamdas[idx], size=nr_of_itt_draws*4))))
                if (np.sum(itts) < minT):
                    itts = np.hstack((itts, np.array(gamma.rvs(a=ks[idx], loc=ks[idx]*lamdas[idx], size=nr_of_itt_draws*800))))
                if (np.sum(itts) < minT):
                    print("...not enough inter-transaction times sampled! cust: {}, draw: {}, {} < {}".format(i, idx, np.sum(itts), minT))
                x_holdout[idx, i] = np.sum(np.cumsum(itts) < minT)
            idx += 1
        if (np.any(~alive)):
            x_holdout[~alive, i] = 0
    return x_holdout

def PActive(x_holdout_draws):
    nr_of_cust = x_holdout_draws.shape[1]
    p_alive = np.zeros(shape=(nr_of_cust))    
    for i in range(0, nr_of_cust):
        cd = x_holdout_draws[:,i]
        p_alive[i] = np.mean(cd[cd > 0])
    return p_alive



In [ ]:
# Main routine
parse_dates = ['first']
g_cbs_dataset = '{}/cbs.csv'.format(g_datafolder)
df = load_dataset(g_cbs_dataset, parse_dates=parse_dates)
covariates, covars, K = set_regressors(df, covariates=["first_sales"])
hyper_prior = set_hyperpriors(K)
draws = run_single_chain(df, covariates=covariates, K=K, hyper_prior=hyper_prior, nsample=500, nburnin=500, nskip=10)
#x_holdout_draws = DrawFutureTransactions(df, draws, sample_size=None)
#df['x_predicted'] = np.mean(x_holdout_draws, axis=0)
#p_alive = PActive(x_holdout_draws)
#df['palive'] = p_alive


In [ ]:
draws['coeff']

In [ ]:
x_holdout_draws = DrawFutureTransactions(df, draws, sample_size=None)
df['x_predicted'] = np.mean(x_holdout_draws, axis=0)
p_alive = PActive(x_holdout_draws)
df['palive'] = p_alive

In [ ]:
df.head()

In [ ]:
np.mean(df['x_holdout'] - df['x_predicted'])

In [ ]:
plt.scatter(df.x_holdout.values, df.x_predicted)

In [ ]:
df[['x_cal', 'x_holdout', 'x_predicted']].groupby(['x_cal']).mean()